# TruBot

Example setup and monitoring of a conversational bot with context made up of the TruEra website.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Uncomment for more debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

In [ ]:
from pprint import PrettyPrinter
from typing import Sequence

from IPython.display import JSON
# imports from langchain to build app
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import LLMChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.prompts.chat import PromptTemplate
from langchain.vectorstores import DocArrayHnswSearch
import numpy as np

# Imports main tools:
from trulens_eval import Feedback
from trulens_eval import Huggingface
from trulens_eval import Query
from trulens_eval import Tru
from trulens_eval import tru_feedback
from trulens_eval import TruChain
from trulens_eval.keys import *
from trulens_eval.tru_db import Query
from trulens_eval.tru_db import Record
from trulens_eval.tru_feedback import Feedback
from trulens_eval.tru_feedback import Huggingface
from trulens_eval.util import Step
from trulens_eval.utils.langchain import WithFeedbackFilterDocuments

# if using Pinecone vectordb:
# from langchain.vectorstores import Pinecone
# import pinecone

tru = Tru()
pp = PrettyPrinter()

tru.reset_database()

In [ ]:
model_name = "gpt-3.5-turbo"
chain_id = "TruBot"

# Pinecone configuration if using pinecone.
# pinecone.init(
#    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
#    environment=PINECONE_ENV  # next to api key in console
#)
#docsearch = Pinecone.from_existing_index(
#    index_name="llmdemo", embedding=embedding
#)

# LLM for completing prompts, and other tasks.
llm = OpenAI(temperature=0, max_tokens=256)

# Construct feedback functions.

hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on(
    prompt=Query.RecordInput, response=Query.RecordOutput
)

# Question/statement relevance between question and each context chunk.
f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance).on(
    question=Query.RecordInput,
    statement=Query.Record.chain.combine_docs_chain._call.args.inputs.
    input_documents[:].page_content
).aggregate(np.min)

def new_conversation(
    lang_prompt_fix: bool = False,
    context_prompt_fix: bool = False,
    context_filter_fix: bool = False,
    feedbacks: Sequence[Feedback] = None
):
    """
    Create a chain for a new conversation (blank memory). Set flags to enable
    adjustments to prompts or add context filtering.
    """
    
    assert not(lang_prompt_fix and context_prompt_fix), "Cannot use both prompt fixes at the same time."

    # Embedding needed for Pinecone vector db.
    embedding = OpenAIEmbeddings(model='text-embedding-ada-002')  # 1536 dims

    # Conversation memory.
    memory = ConversationSummaryBufferMemory(
        max_token_limit=650,
        llm=llm,
        memory_key="chat_history",
        output_key='answer'
    )

    # Pinecone alternative. Requires precomputed 'hnswlib_truera' folder.
    docsearch = DocArrayHnswSearch.from_params(
        embedding=embedding,
        work_dir='hnswlib_trubot',
        n_dim=1536,
        max_elements=1024
    )
    retriever = docsearch.as_retriever()

    # Better contexts fix, filter contexts with relevance:
    if context_filter_fix: 
        retriever = WithFeedbackFilterDocuments.of_retriever(
            retriever=retriever, feedback=f_qs_relevance, threshold = 0.5
        )

    # Conversational chain puts it all together.
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        return_source_documents=True,
        memory=memory,
        get_chat_history=lambda a: a,
        max_tokens_limit=4096
    )

    # Need to copy these otherwise various chains will feature templates that
    # point to the same objects.
    chain.combine_docs_chain.llm_chain.prompt = \
        chain.combine_docs_chain.llm_chain.prompt.copy()
    chain.combine_docs_chain.document_prompt = \
        chain.combine_docs_chain.document_prompt.copy()

    # Language mismatch fix:
    if lang_prompt_fix:
        chain.combine_docs_chain.llm_chain.prompt.template = \
            "Use the following pieces of context to answer the question at the end " \
            "in the same language as the question. If you don't know the answer, " \
            "just say that you don't know, don't try to make up an answer.\n\n" \
            "{context}\n\n" \
            "Question: {question}\n" \
            "Helpful Answer: "

    # Poor contexts fix using prompts:
    elif context_prompt_fix:
        chain.combine_docs_chain.llm_chain.prompt.template = \
            "Use only the relevant contexts to answer the question at the end " \
            ". Some pieces of context may not be relevant. If you don't know the answer, " \
            "just say that you don't know, don't try to make up an answer.\n\n" \
            "Contexts: \n{context}\n\n" \
            "Question: {question}\n" \
            "Helpful Answer: "
        chain.combine_docs_chain.document_prompt.template = "\tContext: {page_content}"

    # Trulens instrumentation.
    tc = Tru().Chain(chain=chain, feedbacks=feedbacks, verbose=True)

    return tc

In [ ]:
# Instantiate a chain

tc = new_conversation(
    #feedbacks=[f_lang_match], # any feedbacks specified here will be executed automatically
    context_filter_fix=True
)

In [ ]:
# Call the chain

res, record = tc.call_with_record("Who is Shayak?")

In [ ]:
record.dict()

In [ ]:
# Evaluate a feedback function manually.

fres = f_qs_relevance.run(chain=tc, record=record)

In [ ]:
fres.dict()